In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import StratifiedKFold

from utilities import load_data

In [2]:
X,y = load_data(100, amplitude=10)

In [3]:
X.shape #2n samples of 400x400 pixels, with a depth of 1 (greyscale type)

(200, 400, 400, 1)

In [4]:
skfold = StratifiedKFold(n_splits=5)

In [6]:
acc_per_fold = []
loss_per_fold = []

for train, test in skfold.split(X,y):
    
    #define the model
    model = keras.Sequential([
        
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=X.shape[1:]),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
        
    keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=X.shape[1:]),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
        
                
    keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=X.shape[1:]),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
        
                
    keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=X.shape[1:]),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
        
    keras.layers.Flatten(),
    keras.layers.Dense(units=512, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    #compile the model
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    
    #fit the model
    history = model.fit(X[train], y[train], epochs=30, batch_size=20)
    
    #evaluate the model
    scores = model.evaluate(X[test], y[test])
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    print("-----------------------------------")



Epoch 1/30
8/8 [==============================] - 15s 2s/step - loss: 1.0894 - acc: 0.5125
Epoch 2/30
8/8 [==============================] - 15s 2s/step - loss: 0.6972 - acc: 0.4812
Epoch 3/30
8/8 [==============================] - 15s 2s/step - loss: 0.6860 - acc: 0.4625
Epoch 4/30
8/8 [==============================] - 15s 2s/step - loss: 0.6621 - acc: 0.5688
Epoch 5/30
8/8 [==============================] - 15s 2s/step - loss: 0.5756 - acc: 0.8188
Epoch 6/30
8/8 [==============================] - 15s 2s/step - loss: 0.5861 - acc: 0.7250
Epoch 7/30
8/8 [==============================] - 15s 2s/step - loss: 0.4906 - acc: 0.8375
Epoch 8/30
8/8 [==============================] - 15s 2s/step - loss: 0.5009 - acc: 0.8125
Epoch 9/30
8/8 [==============================] - 15s 2s/step - loss: 0.4411 - acc: 0.8062
Epoch 10/30
8/8 [==============================] - 15s 2s/step - loss: 0.3404 - acc: 0.9375
Epoch 11/30
8/8 [==============================] - 15s 2s/step - loss: 0.4085 - acc: 0.90

8/8 [==============================] - 15s 2s/step - loss: 0.1522 - acc: 0.9312
Epoch 28/30
8/8 [==============================] - 15s 2s/step - loss: 0.0445 - acc: 1.0000
Epoch 29/30
8/8 [==============================] - 15s 2s/step - loss: 0.0086 - acc: 1.0000
Epoch 30/30
2/2 [==============================] - 0s 100ms/step - loss: 0.7586 - acc: 0.4500
-----------------------------------
Epoch 1/30
8/8 [==============================] - 15s 2s/step - loss: 1.8049 - acc: 0.4437
Epoch 2/30
8/8 [==============================] - 15s 2s/step - loss: 0.7061 - acc: 0.5562
Epoch 3/30
8/8 [==============================] - 15s 2s/step - loss: 0.6805 - acc: 0.5938
Epoch 4/30
8/8 [==============================] - 15s 2s/step - loss: 0.6873 - acc: 0.7188
Epoch 5/30
8/8 [==============================] - 15s 2s/step - loss: 0.6678 - acc: 0.6812
Epoch 6/30
8/8 [==============================] - 15s 2s/step - loss: 0.5888 - acc: 0.6500
Epoch 7/30
8/8 [==============================] - 15s 2s/ste

In [7]:
acc_per_fold

[47.49999940395355,
 42.500001192092896,
 44.999998807907104,
 42.500001192092896,
 52.49999761581421]

In [ ]:
#model.save('CNN_1.h5')